# Inference with `google/gemma-7b-it`. Fine-tuned using PEFT QLoRA

## Purpose
This notebook performs inference using the `google/gemma-7b-it` language model from Hugging Face's `transformers` library, fine-tuned using **PEFT (Parameter-Efficient Fine-Tuning)** with **QLoRA** (Quantized Low Rank Adaptation). The test dataset consists of 1363 records, which are fetched via an HTTP call to the **Palaven API**—a custom API developed exclusively for this thesis. The generated responses, along with evaluation metrics, are saved back to the Palaven API for future analysis.

## Overview

1. **Declaration of Global Variables**:
    - The notebook initializes all necessary global variables, including paths, API credentials, batch sizes, and any configuration settings required for inference and evaluation.

2. **Loading the Fine-tuned Model and Tokenizer**:
    - The `google/gemma-7b-it` model and its corresponding tokenizer are loaded from the Hugging Face library (`transformers`).
    - The model has been fine-tuned using **PEFT** with **QLoRA**, allowing for efficient adaptation to the specific task with reduced memory requirements.

3. **Inference Execution**:
    - **Dataset fetching**: The test dataset, consisting of 1363 records, is retrieved from the cloud via the **Palaven API** using an HTTP request.
    - **Batch processing**: The dataset is processed in batches, with each batch containing 50 instructions. This ensures efficient use of memory and processing resources, especially when working with the fine-tuned model.
    - **Model Inference**: For each instruction in the dataset, the fine-tuned `google/gemma-7b-it` model, enhanced by the RAG component, generates responses.
    - **Saving responses**: All generated responses are persisted back to the Palaven API for future evaluation and analysis.

4. **Evaluation Metrics**:
    - **BERTScore**: Precision, Recall, and F1 scores are calculated for each batch to measure the semantic similarity between the generated text and the ground truth.
    - **ROUGE**: ROUGE 1, ROUGE 2, and ROUGE L are computed to evaluate n-gram overlap, with Precision, Recall, and F1 calculated for each metric.
    - **BLEU**: The BLEU score is calculated for each batch, assessing the fluency and correctness of the generated text based on n-gram precision.
    - **Persisting metrics**: All computed metrics are persisted to the cloud via the Palaven API, allowing for deeper analysis and comparisons with other experiments (e.g., out-of-the-box vs. fine-tuned).


## Key Sections
### 1. Declaration of global variables

| **Variable**             | **Description**                                                                                   | **Value**                                                   |
|--------------------------|---------------------------------------------------------------------------------------------------|-------------------------------------------------------------|
| `hface_read_token`        | Access token for reading from Hugging Face, retrieved from user data.                             | `userdata.get('hface-read-token')`                          |
| `palaven_base_url`        | Base URL for the Palaven API, retrieved from user data.                                           | `userdata.get('palaven-base-url')`                          |
| `batch_size`              | Batch size for inference and evaluation execution.                                                | `50`                                                        |
| `dataset_id`              | Unique identifier for the dataset used in evaluation.                                             | `'F0444B12-5485-4299-B03B-3BDB6D4A2578'`                    |
| `evaluation_session_id`   | Unique identifier for the current evaluation session.                                             | `'EB9C5839-7B20-4D7D-B3F7-17528180676D'`                    |
| `llm_palaven_name`        | Name of the model used in Palaven.                                                 | `'google-gemma'`                                            |
| `llm_model_name`          | Full name of the model used from Hugging Face.                                                    | `'google/gemma-7b-it'`                                      |
| `device_info`             | Information about the device where the inferences and evaluations are executed.                   | `'GPU A100'`                                                |


In [2]:
from google.colab import userdata
from palaven_api_v2 import PalavenApi

hface_read_token = userdata.get('hface-read-token')
palaven_base_url = userdata.get('palaven-base-url')

batch_size = 50
dataset_id = 'F0444B12-5485-4299-B03B-3BDB6D4A2578'
evaluation_session_id = 'EB9C5839-7B20-4D7D-B3F7-17528180676D'
llm_palaven_name = 'google-gemma'
llm_model_name = 'google/gemma-7b-it'
llm_palaven_peft = '/content/drive/MyDrive/Colab Notebooks/palaven/gemma-7b-it-palaven'
device_info = 'GPU A100'

palaven_api = PalavenApi(palaven_base_url)

### 2. Load the base model and the tokenizer

In [ ]:
!pip install -U transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 73.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(llm_model_name, token=hface_read_token)
base_model = AutoModelForCausalLM.from_pretrained(llm_model_name, device_map="auto", token=hface_read_token)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

### 2.1. Load the PEFT QLoRA finetuned adapters and merge them with the base model google/gemma-7b-it

In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
from peft import PeftModel

peft_model = PeftModel.from_pretrained(base_model, llm_palaven_peft, token=hface_read_token)
model = peft_model.merge_and_unload()
model.to("cuda")

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear(in_features=24576, out_features=3072, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((3072,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((3072,), eps=1

### 3. Inference Execution

In [ ]:
import time
import torch

def get_chat_completion(instruction_prompt, instruction_id, instruction_count):

  start_time = time.time()

  tokenized_instruction = tokenizer(instruction_prompt, return_tensors="pt").to("cuda")

  with torch.no_grad():
    chat_completion_result = model.generate(**tokenized_instruction, max_new_tokens=1245)

  chat_completion = tokenizer.decode(chat_completion_result[0])

  end_time = time.time()
  elapsed_time = end_time - start_time

  print(f'{instruction_count} - LLM-ChatCompletion. InstructionId: {instruction_id},  Elapsed-Time: {elapsed_time:.2f} seconds')

  return chat_completion, elapsed_time

In [ ]:
def add_instruction_to_df(df, instruction_id, instruction):
    df.loc[instruction_id, 'instruction'] = instruction
    df.loc[instruction_id, 'chat_completion'] = None
    df.loc[instruction_id, 'elapsed_time'] = None

In [ ]:
import pandas as pd
import numpy as np
import json

data_shape = {
    'instruction_id': [],
    'instruction': [],
    'chat_completion': [],
    'elapsed_time': []
}

for batch_number in range(1, 29):

  print(f'Start fetching batch {batch_number}')

  instructions = palaven_api.fetch_instruction_test_dataset(evaluation_session_id, batch_number=batch_number, evaluation_exercise='llmfinetuned')

  print(f'Start fetching batch done...')

  llm_responses_df = pd.DataFrame(data_shape)
  llm_responses_df.set_index('instruction_id', inplace=True)

  for item in instructions:
    add_instruction_to_df(llm_responses_df, item['instructionId'], item['instruction'])

  instruction_count = 1

  for index, row in llm_responses_df.iterrows():
    instruction = llm_responses_df.loc[index, 'instruction']

    instruction_prompt = f"""
      <start_of_turn>user
      Answer the following question in a concise and informative manner. The question is written in Spanish language, then answer in Spanish language.
      {instruction}<end_of_turn>
      <start_of_turn>model
    """

    instruction_id = int(index)

    chat_completion, elapsed_time = get_chat_completion(instruction_prompt, instruction_id, instruction_count)
    palaven_api.save_model_response(evaluation_session_id, 'llmfinetuned', batch_number, instruction_id, chat_completion, elapsed_time)

    instruction_count += 1

Start fetching batch 1
Start fetching batch done...


<ipython-input-15-9ad68d3380dc>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '¿Bajo qué circunstancias los residentes en el extranjero deben pagar impuesto sobre la renta en México según el Artículo 1 de la Ley del Impuesto Sobre la Renta del 2024?' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[instruction_id, 'instruction'] = instruction


1 - LLM-ChatCompletion. InstructionId: 3878,  Elapsed-Time: 3.89 seconds
Palaven.SaveResponse. InstructionId: 3878,  Elapsed-Time: 0.51 seconds
2 - LLM-ChatCompletion. InstructionId: 3880,  Elapsed-Time: 1.17 seconds
Palaven.SaveResponse. InstructionId: 3880,  Elapsed-Time: 0.25 seconds
3 - LLM-ChatCompletion. InstructionId: 3882,  Elapsed-Time: 1.96 seconds
Palaven.SaveResponse. InstructionId: 3882,  Elapsed-Time: 0.17 seconds
4 - LLM-ChatCompletion. InstructionId: 3884,  Elapsed-Time: 3.06 seconds
Palaven.SaveResponse. InstructionId: 3884,  Elapsed-Time: 0.15 seconds
5 - LLM-ChatCompletion. InstructionId: 3886,  Elapsed-Time: 2.51 seconds
Palaven.SaveResponse. InstructionId: 3886,  Elapsed-Time: 0.20 seconds
6 - LLM-ChatCompletion. InstructionId: 3888,  Elapsed-Time: 1.92 seconds
Palaven.SaveResponse. InstructionId: 3888,  Elapsed-Time: 0.24 seconds
7 - LLM-ChatCompletion. InstructionId: 3894,  Elapsed-Time: 4.36 seconds
Palaven.SaveResponse. InstructionId: 3894,  Elapsed-Time: 0.19 

<ipython-input-15-9ad68d3380dc>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '¿Para calcular la modificación en el valor de bienes o operaciones en un periodo de un mes, qué factor de ajuste se utiliza de acuerdo al Artículo 6 de la Ley del Impuesto Sobre la Renta del 2024?' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[instruction_id, 'instruction'] = instruction


1 - LLM-ChatCompletion. InstructionId: 4017,  Elapsed-Time: 3.19 seconds
Palaven.SaveResponse. InstructionId: 4017,  Elapsed-Time: 0.15 seconds
2 - LLM-ChatCompletion. InstructionId: 4025,  Elapsed-Time: 5.53 seconds
Palaven.SaveResponse. InstructionId: 4025,  Elapsed-Time: 0.25 seconds
3 - LLM-ChatCompletion. InstructionId: 4027,  Elapsed-Time: 1.47 seconds
Palaven.SaveResponse. InstructionId: 4027,  Elapsed-Time: 0.15 seconds
4 - LLM-ChatCompletion. InstructionId: 4029,  Elapsed-Time: 1.79 seconds
Palaven.SaveResponse. InstructionId: 4029,  Elapsed-Time: 0.19 seconds
5 - LLM-ChatCompletion. InstructionId: 4031,  Elapsed-Time: 1.68 seconds
Palaven.SaveResponse. InstructionId: 4031,  Elapsed-Time: 0.17 seconds
6 - LLM-ChatCompletion. InstructionId: 4033,  Elapsed-Time: 3.17 seconds
Palaven.SaveResponse. InstructionId: 4033,  Elapsed-Time: 0.17 seconds
7 - LLM-ChatCompletion. InstructionId: 4035,  Elapsed-Time: 2.61 seconds
Palaven.SaveResponse. InstructionId: 4035,  Elapsed-Time: 0.26 

...

1 - LLM-ChatCompletion. InstructionId: 7433,  Elapsed-Time: 2.55 seconds
Palaven.SaveResponse. InstructionId: 7433,  Elapsed-Time: 0.17 seconds
2 - LLM-ChatCompletion. InstructionId: 7435,  Elapsed-Time: 1.79 seconds
Palaven.SaveResponse. InstructionId: 7435,  Elapsed-Time: 0.16 seconds
3 - LLM-ChatCompletion. InstructionId: 7437,  Elapsed-Time: 1.51 seconds
Palaven.SaveResponse. InstructionId: 7437,  Elapsed-Time: 0.19 seconds
4 - LLM-ChatCompletion. InstructionId: 7439,  Elapsed-Time: 1.91 seconds
Palaven.SaveResponse. InstructionId: 7439,  Elapsed-Time: 0.17 seconds
5 - LLM-ChatCompletion. InstructionId: 7441,  Elapsed-Time: 1.51 seconds
Palaven.SaveResponse. InstructionId: 7441,  Elapsed-Time: 0.23 seconds
6 - LLM-ChatCompletion. InstructionId: 7451,  Elapsed-Time: 4.41 seconds
Palaven.SaveResponse. InstructionId: 7451,  Elapsed-Time: 0.17 seconds
7 - LLM-ChatCompletion. InstructionId: 7453,  Elapsed-Time: 1.86 seconds
Palaven.SaveResponse. InstructionId: 7453,  Elapsed-Time: 0.16 

<ipython-input-15-9ad68d3380dc>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Sumariza el artículo 169 de la Ley del Impuesto Sobre la Renta de 2024' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[instruction_id, 'instruction'] = instruction


1 - LLM-ChatCompletion. InstructionId: 7577,  Elapsed-Time: 4.97 seconds
Palaven.SaveResponse. InstructionId: 7577,  Elapsed-Time: 0.21 seconds
2 - LLM-ChatCompletion. InstructionId: 7579,  Elapsed-Time: 1.42 seconds
Palaven.SaveResponse. InstructionId: 7579,  Elapsed-Time: 0.19 seconds
3 - LLM-ChatCompletion. InstructionId: 7581,  Elapsed-Time: 2.01 seconds
Palaven.SaveResponse. InstructionId: 7581,  Elapsed-Time: 0.15 seconds
4 - LLM-ChatCompletion. InstructionId: 7583,  Elapsed-Time: 0.90 seconds
Palaven.SaveResponse. InstructionId: 7583,  Elapsed-Time: 0.17 seconds
5 - LLM-ChatCompletion. InstructionId: 7585,  Elapsed-Time: 1.53 seconds
Palaven.SaveResponse. InstructionId: 7585,  Elapsed-Time: 0.15 seconds
6 - LLM-ChatCompletion. InstructionId: 7587,  Elapsed-Time: 1.91 seconds
Palaven.SaveResponse. InstructionId: 7587,  Elapsed-Time: 0.20 seconds
7 - LLM-ChatCompletion. InstructionId: 7589,  Elapsed-Time: 1.06 seconds
Palaven.SaveResponse. InstructionId: 7589,  Elapsed-Time: 0.26 

<ipython-input-15-9ad68d3380dc>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '¿Los residentes en México y los residentes en el extranjero con establecimiento permanente en el país están obligados a pagar impuestos por los ingresos sujetos a regímenes fiscales preferentes obtenidos a través de entidades extranjeras en las que participen?' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[instruction_id, 'instruction'] = instruction


1 - LLM-ChatCompletion. InstructionId: 7712,  Elapsed-Time: 1.72 seconds
Palaven.SaveResponse. InstructionId: 7712,  Elapsed-Time: 0.16 seconds
2 - LLM-ChatCompletion. InstructionId: 7714,  Elapsed-Time: 1.67 seconds
Palaven.SaveResponse. InstructionId: 7714,  Elapsed-Time: 0.18 seconds
3 - LLM-ChatCompletion. InstructionId: 7716,  Elapsed-Time: 1.85 seconds
Palaven.SaveResponse. InstructionId: 7716,  Elapsed-Time: 0.20 seconds
4 - LLM-ChatCompletion. InstructionId: 7725,  Elapsed-Time: 4.99 seconds
Palaven.SaveResponse. InstructionId: 7725,  Elapsed-Time: 0.23 seconds
5 - LLM-ChatCompletion. InstructionId: 7727,  Elapsed-Time: 2.00 seconds
Palaven.SaveResponse. InstructionId: 7727,  Elapsed-Time: 0.17 seconds
6 - LLM-ChatCompletion. InstructionId: 7729,  Elapsed-Time: 2.03 seconds
Palaven.SaveResponse. InstructionId: 7729,  Elapsed-Time: 0.16 seconds
7 - LLM-ChatCompletion. InstructionId: 7731,  Elapsed-Time: 1.33 seconds
Palaven.SaveResponse. InstructionId: 7731,  Elapsed-Time: 0.17 

### 4. Evaluation metrics

4.1. BERTScore

In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00


In [4]:
import pandas as pd

def build_responses_df():
  dataframe = pd.DataFrame(columns=['instruction_id',
    'evaluation_session_id', 'dataset_id', 'batch_size', 'large_language_model',
    'device_info', 'excercise_type', 'batch_number', 'instruction',
    'response', 'category', 'response_to_evaluate', 'elapsed_time'])

  dataframe['evaluation_session_id'] = dataframe['evaluation_session_id'].astype('object')
  dataframe['dataset_id'] = dataframe['dataset_id'].astype('object')
  dataframe['large_language_model'] = dataframe['large_language_model'].astype('object')
  dataframe['device_info'] = dataframe['device_info'].astype('object')
  dataframe['excercise_type'] = dataframe['excercise_type'].astype('object')
  dataframe['instruction'] = dataframe['instruction'].astype('object')
  dataframe['response'] = dataframe['response'].astype('object')
  dataframe['category'] = dataframe['category'].astype('object')
  dataframe['response_to_evaluate'] = dataframe['response_to_evaluate'].astype('object')

  dataframe.set_index('instruction_id', inplace=True)

  return dataframe


def add_response_to_df(dataframe, response):
  dataframe.loc[response['instructionId'], 'evaluation_session_id'] = response['evaluationSessionId']
  dataframe.loc[response['instructionId'], 'dataset_id'] = response['datasetId']
  dataframe.loc[response['instructionId'], 'batch_size'] = response['batchSize']
  dataframe.loc[response['instructionId'], 'large_language_model'] = response['largeLanguageModel']
  dataframe.loc[response['instructionId'], 'device_info'] = response['deviceInfo']
  dataframe.loc[response['instructionId'], 'excercise_type'] = response['evaluationExercise']
  dataframe.loc[response['instructionId'], 'batch_number'] = response['batchNumber']
  dataframe.loc[response['instructionId'], 'instruction'] = response['instruction']
  dataframe.loc[response['instructionId'], 'response'] = response['response']
  dataframe.loc[response['instructionId'], 'category'] = response['category']
  dataframe.loc[response['instructionId'], 'response_to_evaluate'] = response['llmResponseToEvaluate']
  dataframe.loc[response['instructionId'], 'elapsed_time'] = response['elapsedTime']

In [ ]:
from bert_score import score as bert_score

for batch_number in range(1, 29):

  model_responses_df = build_responses_df()

  print(f'Palaven. Bertscore evaluation. Start processing batch: {batch_number}')

  model_responses = palaven_api.fetch_model_responses(evaluation_session_id, 'llmfinetuned', batch_number)

  for item in model_responses:
    add_response_to_df(model_responses_df, item)

  references = model_responses_df['response'].tolist()
  candidates = model_responses_df['response_to_evaluate'].tolist()

  accuracy, recall, f1 = bert_score(candidates, references, lang='es', verbose=True, device='cuda')

  accuracy = accuracy.tolist()
  recall = recall.tolist()
  f1 = f1.tolist()

  average_accuracy = sum(accuracy) / len(accuracy)
  average_recall = sum(recall) / len(recall)
  average_f1 = sum(f1) / len(f1)

  print(f'Palaven. Bertscore evaluation. Batch: {batch_number}. Average accuracy: {average_accuracy}. Average recall: {average_recall}. Average F1: {average_f1}')

  palaven_api.save_bert_score_metrics(evaluation_session_id, 'llmfinetuned', batch_number, average_accuracy, average_recall, average_f1)

  print(f'Palaven. Bertscore evaluation. Posted batch {batch_number}')

Palaven. Bertscore evaluation. Start processing batch: 1


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.12 seconds, 108.61 sentences/sec
Palaven. Bertscore evaluation. Batch: 28. Average accuracy: 0.7982437473077041. Average recall: 0.79885811989124. Average F1: 0.7972541451454163
Palaven. Bertscore evaluation. Posted batch 28


4.2. ROUGE (1,2-L)

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=2264653c5e22aed77a260be3e3cc12227f1315e324d5e4ac50d94d18a00ab78a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from rouge_score import rouge_scorer

for batch_number in range(1, 29):

  model_responses_df = build_responses_df()

  print(f'Palaven. ROUGE evaluation. Start processing batch: {batch_number}')
  model_responses = palaven_api.fetch_model_responses(evaluation_session_id, 'llmfinetuned', batch_number)

  for item in model_responses:
    add_response_to_df(model_responses_df, item)

  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

  r1_precision, r1_recall, r1_f1 = [], [], []
  r2_precision, r2_recall, r2_f1 = [], [], []
  rL_precision, rL_recall, rL_f1 = [], [], []

  for _, row in model_responses_df.iterrows():
    reference = row['response']
    candidate = row['response_to_evaluate']

    scores = scorer.score(reference, candidate)

    #ROUGE-1
    r1_precision.append(scores['rouge1'].precision)
    r1_recall.append(scores['rouge1'].recall)
    r1_f1.append(scores['rouge1'].fmeasure)

    #ROUGE-2
    r2_precision.append(scores['rouge2'].precision)
    r2_recall.append(scores['rouge2'].recall)
    r2_f1.append(scores['rouge2'].fmeasure)

    #ROUGE-L
    rL_precision.append(scores['rougeL'].precision)
    rL_recall.append(scores['rougeL'].recall)
    rL_f1.append(scores['rougeL'].fmeasure)


  average_r1_precision = sum(r1_precision) / len(r1_precision)
  average_r1_recall = sum(r1_recall) / len(r1_recall)
  average_r1_f1 = sum(r1_f1) / len(r1_f1)

  average_r2_precision = sum(r2_precision) / len(r2_precision)
  average_r2_recall = sum(r2_recall) / len(r2_recall)
  average_r2_f1 = sum(r2_f1) / len(r2_f1)

  average_rL_precision = sum(rL_precision) / len(rL_precision)
  average_rL_recall = sum(rL_recall) / len(rL_recall)
  average_rL_f1 = sum(rL_f1) / len(rL_f1)

  rouge_metrics = []

  rouge_metrics.append({
      'rougeScoreType': 'rouge1',
      'batchNumber': batch_number,
      'precision': average_r1_precision,
      'recall': average_r1_recall,
      'f1': average_r1_f1
  });

  rouge_metrics.append({
      'rougeScoreType': 'rouge2',
      'batchNumber': batch_number,
      'precision': average_r2_precision,
      'recall': average_r2_recall,
      'f1': average_r2_f1
  });

  rouge_metrics.append({
      'rougeScoreType': 'rougeL',
      'batchNumber': batch_number,
      'precision': average_rL_precision,
      'recall': average_rL_recall,
      'f1': average_rL_f1
  });

  print(rouge_metrics)

  print(f'Palaven. ROUGE evaluation finished for batch: {batch_number}.')

  palaven_api.save_rouge_metrics(evaluation_session_id, 'llmfinetuned', batch_number, rouge_metrics)

  print(f'Palaven. Bertscore evaluation. Posted batch {batch_number}')

Palaven. ROUGE evaluation. Start processing batch: 1
[{'rougeScoreType': 'rouge1', 'batchNumber': 1, 'precision': 0.3745757037925945, 'recall': 0.5200344327247403, 'f1': 0.3994700148016524}, {'rougeScoreType': 'rouge2', 'batchNumber': 1, 'precision': 0.20918508754335519, 'recall': 0.26557602773344674, 'f1': 0.22013673991462032}, {'rougeScoreType': 'rougeL', 'batchNumber': 1, 'precision': 0.28583078366517883, 'recall': 0.40565022807253615, 'f1': 0.3035854998657952}]
Palaven. ROUGE evaluation finished for batch: 1.
Palaven. Bertscore evaluation. Posted batch 1
Palaven. ROUGE evaluation. Start processing batch: 2
[{'rougeScoreType': 'rouge1', 'batchNumber': 2, 'precision': 0.4215934871983799, 'recall': 0.48263873944799485, 'f1': 0.4245934431197033}, {'rougeScoreType': 'rouge2', 'batchNumber': 2, 'precision': 0.24555796082318845, 'recall': 0.27986940522397963, 'f1': 0.24894575912864922}, {'rougeScoreType': 'rougeL', 'batchNumber': 2, 'precision': 0.3324951399514729, 'recall': 0.38245134230

### 4.3. BLEU

In [5]:
from nltk.translate.bleu_score import sentence_bleu

for batch_number in range(1, 29):

    model_responses_df = build_responses_df()

    print(f'Palaven. BLEU evaluation. Start processing batch: {batch_number}')

    model_responses = palaven_api.fetch_model_responses(evaluation_session_id, 'llmfinetuned', batch_number)

    for item in model_responses:
        add_response_to_df(model_responses_df, item)

    references = model_responses_df['response'].tolist()
    candidates = model_responses_df['response_to_evaluate'].tolist()

    bleu_scores = []

    for candidate, reference in zip(candidates, references):
        reference_tokens = [reference.split()]
        candidate_tokens = candidate.split()
        bleu_score = sentence_bleu(reference_tokens, candidate_tokens)
        bleu_scores.append(bleu_score)

    average_bleu = sum(bleu_scores) / len(bleu_scores)

    print(f'Palaven. BLEU evaluation. Batch: {batch_number}. Average BLEU score: {average_bleu}')

    palaven_api.save_bleu_score_metrics(evaluation_session_id, 'llmfinetuned', batch_number, average_bleu)

    print(f'Palaven. BLEU evaluation. Posted batch {batch_number}')

Palaven. BLEU evaluation. Start processing batch: 1


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Palaven. BLEU evaluation. Batch: 1. Average BLEU score: 0.11486506402694947
Palaven. BLEU evaluation. Posted batch 1
Palaven. BLEU evaluation. Start processing batch: 2


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Palaven. BLEU evaluation. Batch: 2. Average BLEU score: 0.15236380874337024
Palaven. BLEU evaluation. Posted batch 2
Palaven. BLEU evaluation. Start processing batch: 3


...

Palaven. BLEU evaluation. Batch: 26. Average BLEU score: 0.13396212907490831
Palaven. BLEU evaluation. Posted batch 26
Palaven. BLEU evaluation. Start processing batch: 27


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Palaven. BLEU evaluation. Batch: 27. Average BLEU score: 0.12628881974102202
Palaven. BLEU evaluation. Posted batch 27
Palaven. BLEU evaluation. Start processing batch: 28


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Palaven. BLEU evaluation. Batch: 28. Average BLEU score: 0.2671366620441368
Palaven. BLEU evaluation. Posted batch 28
